In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from datetime import datetime
import math
import pandas as pd
import xgboost as xgb
import wandb
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [70]:
def merge_csv_files(directory):
    # List all CSV files in the directory
    csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]

    # Initialize an empty DataFrame to store the merged data
    merged_df = pd.DataFrame()
    count=1
    # Flag to indicate whether to include headers
    include_headers = True
    flag=0 if include_headers else None
    # Iterate over each CSV file and merge into the DataFrame
    for file in csv_files:
        file_path = os.path.join(directory, file)
        # Skip headers for all files except the first one
        df = pd.read_csv(file_path, header=flag)
        
        merged_df = pd.concat([merged_df, df], ignore_index=True)
        include_headers = False  # Headers only for the first file
        if count%5==0:
            print(count/len(csv_files))
        count+=1

    return merged_df

In [71]:

# def merge_csv_files(directory):
#     # List all CSV files in the directory
#     csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]

#     # Initialize an empty DataFrame to store the merged data
#     merged_df = pd.DataFrame()
#     count=1
#     # Flag to indicate whether to include headers
#     include_headers = True
#     flag=0 if include_headers else None
#     # Iterate over each CSV file and merge into the DataFrame
#     for file in csv_files:
#         file_path = os.path.join(directory, file)
#         # Skip headers for all files except the first one
#         df = pd.read_csv(file_path, header=flag)
#         df=df[df['area_code'].str.contains('VIC', na=False) | df['area_code'].isnull()]
#         merged_df = pd.concat([merged_df, df], ignore_index=True)
#         include_headers = False  # Headers only for the first file
#         if count%5==0:
#             print(count/len(csv_files))
#         count+=1

#     return merged_df

# # Specify the directory containing CSV files
# csv_directory = './weather/obs/'

# # Merge CSV files and store the merged DataFrame
# obs_df = merge_csv_files(csv_directory)
# output_file_path = 'merged_weather_obs.pkl'  # Adjust the filename and path as needed
# obs_df.to_pickle(output_file_path)


In [72]:
obs_df_=pd.read_pickle("./merged_weather_obs.pkl")
station_df_1=pd.read_csv("./StationData_1.csv")
station_df_2=pd.read_csv("./StationData_2.csv")
station_df_3=pd.read_csv("./StationData_3.csv")
station_df = pd.concat([station_df_2, station_df_2, station_df_3], ignore_index=True)
station_df.drop_duplicates(subset=["WMO_NUM","station_number","LATITUDE","LONGITUDE"],keep='first', inplace=True)
obs_df=obs_df_.merge(station_df[["station_number","REGION"]], on='station_number', how='left')
obs_df.loc[~(obs_df["area_code"].isna()),"REGION"]=obs_df.loc[~(obs_df["area_code"].isna()),"area_code"].str[:3]
obs_df.rename(columns={'REGION': 'region'}, inplace=True)

In [73]:
(obs_df.loc[~(obs_df["area_code"].isna()),"region"]==obs_df_.loc[~(obs_df["area_code"].isna()),"area_code"].str[:3]).all()

True

In [74]:
obs_df_VIC=obs_df[obs_df["region"]=="VIC"]
obs_df_VIC.loc[:,"valid_start"]=pd.to_datetime(obs_df_VIC['valid_start'], unit='s')
obs_df_VIC.loc[:,"valid_end"]=pd.to_datetime(obs_df_VIC['valid_end'], unit='s')
obs_df_VIC.loc[:,"qc_valid_start"]=pd.to_datetime(obs_df_VIC['qc_valid_start'], unit='s')
obs_df_VIC.loc[:,"qc_valid_end"]=pd.to_datetime(obs_df_VIC['qc_valid_end'], unit='s')
obs_df_VIC.loc[:,"qc_valid_minutes_start"]=pd.to_datetime(obs_df_VIC['qc_valid_minutes_start'], unit='s')
obs_df_VIC.loc[:,"qc_valid_minutes_end"]=pd.to_datetime(obs_df_VIC['qc_valid_minutes_end'], unit='s')
obs_df_VIC=obs_df_VIC[obs_df_VIC.parameter.isin(["AIR_TEMP","AIR_TEMP_MAX","AIR_TEMP_MIN","PRCP","MaxT","MinT","T","Precip"])]
replacement_mapping = {
    'MaxT': 'AIR_TEMP_MAX',
    'MinT': 'AIR_TEMP_MIN',
    'T': 'AIR_TEMP',
    'Precip': 'PRCP'
}

obs_df_VIC['parameter'] = obs_df_VIC['parameter'].replace(replacement_mapping)

C:\Users\murca\AppData\Local\Temp\ipykernel_2092\2369487298.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs_df_VIC.loc[:,"valid_start"]=pd.to_datetime(obs_df_VIC['valid_start'], unit='s')
C:\Users\murca\AppData\Local\Temp\ipykernel_2092\2369487298.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs_df_VIC.loc[:,"valid_end"]=pd.to_datetime(obs_df_VIC['valid_end'], unit='s')
c:\Users\murca\anaconda3\envs\myenv\lib\site-packages\pandas\core\tools\datetimes.py:557: RuntimeWarning: invalid value enco

In [75]:

def custom_agg(x):
    print(x)
    if (x['parameter']=='PRCP').any():
        return (x['value'] / (x['valid_end'] - x['valid_start'])).mean()
    else:
        return x['value'].mean()
df_obs=pd.DataFrame((obs_df_VIC.groupby(["valid_start","parameter"]).agg({"value":"mean"}))).reset_index()
df_obs=df_obs.pivot(index='valid_start', columns='parameter', values='value').reset_index()

In [76]:
df_obs.head()

parameter,valid_start,AIR_TEMP,AIR_TEMP_MAX,AIR_TEMP_MIN,PRCP
0,2015-04-30 14:00:00,9.085135,9.381081,8.331081,0.0
1,2015-04-30 15:00:00,8.638356,9.053425,7.994521,0.0
2,2015-04-30 16:00:00,8.368493,8.739726,7.736986,0.0
3,2015-04-30 17:00:00,8.087671,8.541096,7.498630,0.0
4,2015-04-30 18:00:00,7.967123,8.350685,7.397260,0.0


In [77]:
demand_dir="./Demand/"
demand_df = merge_csv_files(demand_dir)
# Convert 'SETTLEMENTDATE' to a datetime object
demand_df['SETTLEMENTDATE'] = pd.to_datetime(demand_df['SETTLEMENTDATE'])
# Check if 'SETTLEMENTDATE' is at half-hour
half_hour_mask = demand_df['SETTLEMENTDATE'].dt.minute == 30

# Add a date offset to the half-hour timestamps
demand_df.loc[half_hour_mask, 'SETTLEMENTDATE'] = demand_df['SETTLEMENTDATE'] + pd.DateOffset(minutes=30)

# Group by the rounded 'SETTLEMENTDATE' and sum the 'TOTALDEMAND' values
hourly_demand_df = demand_df.groupby(demand_df['SETTLEMENTDATE']).agg({'TOTALDEMAND': 'sum'}).reset_index()
hourly_demand_df.rename(columns={"SETTLEMENTDATE":"valid_start","TOTALDEMAND":"total_demand"},inplace=True)


0.13513513513513514
0.2702702702702703
0.40540540540540543
0.5405405405405406
0.6756756756756757
0.8108108108108109
0.9459459459459459


In [78]:
df=df_obs.merge(hourly_demand_df,how="left",on="valid_start")


In [79]:
df.head()

,valid_start,AIR_TEMP,AIR_TEMP_MAX,AIR_TEMP_MIN,PRCP,total_demand
0,2015-04-30 14:00:00,9.085135,9.381081,8.331081,0.0,11174.56
1,2015-04-30 15:00:00,8.638356,9.053425,7.994521,0.0,11179.93
2,2015-04-30 16:00:00,8.368493,8.739726,7.736986,0.0,11205.02
3,2015-04-30 17:00:00,8.087671,8.541096,7.498630,0.0,11547.60
4,2015-04-30 18:00:00,7.967123,8.350685,7.397260,0.0,12290.58


In [80]:
df_new = df.copy()

In [81]:
special_holidays = [
    "1-1",  # New Year's Day
    "2-1",  # New Year's Day (additional day)
    "26-1",  # Australia Day
    "13-3",  # Labour Day
    "7-4",  # Good Friday
    "8-4",  # Easter Saturday
    # Since you did not provide a date for "Easter Sunday", I'm omitting it.
    "10-4",  # Easter Monday
    "25-4",  # Anzac Day
    "12-6",  # King's Birthday
    # Omitting the AFL Grand Final date since it's TBC.
    "7-11",  # Melbourne Cup
    "25-12",  # Christmas Day
    "26-12"  # Boxing Day
]


# Helper function to determine the season based on the month
def get_season(month):
    if 12 <= month <= 2:
        return 'summer'
    elif 3 <= month <= 5:
        return 'autumn'
    elif 6 <= month <= 8:
        return 'winter'
    elif 9 <= month <= 11:
        return 'spring'

def periodic_encoding(x, max_val):
    # Calculate sine and cosine values and explicitly cast them as float
    sin = np.sin(2 * np.pi * x / max_val).astype('float64')
    cos = np.cos(2 * np.pi * x / max_val).astype('float64')
    return sin, cos

# The main function to create the dataset
def create_forecasting_dataset(df, n_lag, n_next, special_holidays, 
                               day_embedding='dummy', week_embedding='dummy', season_embedding='dummy'):
    # Convert valid_start to datetime and extract components
    df['valid_start'] = pd.to_datetime(df['valid_start'])
    df['hour'] = df['valid_start'].dt.hour
    df['day'] = df['valid_start'].dt.day
    df['month'] = df['valid_start'].dt.month
    df['year'] = df['valid_start'].dt.year
    df['day_of_week'] = df['valid_start'].dt.dayofweek
    df['week_of_year'] = df['valid_start'].dt.isocalendar().week.astype('float64')
    df['season'] = df['month'].apply(get_season)
    
    # Identify special holidays
    df['date_str'] = df['valid_start'].dt.strftime('%d-%m')
    df['is_special_day'] = df['date_str'].isin(special_holidays).astype(int)
    
    # Periodic or Dummy Encoding
    if day_embedding == 'periodic':
        day_sin, day_cos = periodic_encoding(df['day_of_week'], 7)
        df['day_sin'] = day_sin
        df['day_cos'] = day_cos
    else:
        df = pd.get_dummies(df, columns=['day_of_week'])

    if week_embedding == 'periodic':
        # Encode the week_of_year column
        week_sin, week_cos = periodic_encoding(df['week_of_year'], 52)
        
        # Assign the encoded values ensuring they're in the correct float format
        df['week_sin'] = week_sin.astype('float64')
        df['week_cos'] = week_cos.astype('float64')
    else:
        # If not 'periodic', apply dummy encoding
        df = pd.get_dummies(df, columns=['week_of_year'])

    if season_embedding == 'periodic':
        seasons_map = {'spring': 0, 'summer': 1, 'autumn': 2, 'winter': 3}
        df['season'] = df['season'].map(seasons_map)
        season_sin, season_cos = periodic_encoding(df['season'], 4)
        df['season_sin'] = season_sin
        df['season_cos'] = season_cos
    else:
        df = pd.get_dummies(df, columns=['season'])

    # Create lagged demand values
    for lag in range(1, n_lag + 1):
        df[f'lag_{lag}_hours'] = df['total_demand'].shift(lag)

    # Create future demand values (shifting backwards will bring future values to the present row)
    future_demands = [df['total_demand'].shift(-i) for i in range(n_next)]
    df_future = pd.concat(future_demands, axis=1)
    
    # Ensure there are no NaN values in the future demands (which will be our Y)
    # We also want to make sure there are no NaNs in the lagged features
    mask = df_future.notna().all(axis=1) & df[[f'lag_{lag}_hours' for lag in range(1, n_lag + 1)]].notna().all(axis=1)
    df = df.loc[mask]
    df_future = df_future.loc[mask]

    # Drop columns that are not needed for features
    columns_to_drop = ['valid_start', 'year', 'month', 'day', 'hour', 'date_str', 'total_demand']
    if 'season_mapped' in df.columns:
        columns_to_drop.append('season_mapped')
    df.drop(columns=columns_to_drop, axis=1, inplace=True)
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype(float)
    
    # Ensure that all floating point numbers are in the correct format
    float_cols = df.select_dtypes(include=['float64', 'float32']).columns
    df[float_cols] = df[float_cols].astype('float64')

    # Now both X and Y have the same number of rows
    X = df.values
    Y = df_future.values

    return X, Y
# Example usage
# Load your dataframe from csv or other file format here
# df = pd.read_csv('path_to_your_dataframe.csv')
n_lag = 168  # Number of lag hours
n_next = 8   # Number of hours to forecast
X, Y = create_forecasting_dataset(df_new, n_lag, n_next, special_holidays,
                                  day_embedding='dummy', week_embedding='dummy', season_embedding='dummy')


C:\Users\murca\AppData\Local\Temp\ipykernel_2092\219453476.py:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'lag_{lag}_hours'] = df['total_demand'].shift(lag)
C:\Users\murca\AppData\Local\Temp\ipykernel_2092\219453476.py:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'lag_{lag}_hours'] = df['total_demand'].shift(lag)
C:\Users\murca\AppData\Local\Temp\ipykernel_2092\219453476.py:83: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor p

In [82]:
def evaluate_model(model, X, y):
    # Make predictions on the provided dataset
    predictions = model.predict(X)

    # Calculate MSE and MAPE
    mse = mean_squared_error(y, predictions)
    mape = mean_absolute_percentage_error(y, predictions)

    return mse, mape

In [83]:
wandb.init(project="DI502_PROJECT_new")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: muratcankaplann98. Use `wandb login --relogin` to force relogin


In [84]:
def split_dataset(X, Y, train_size=0.8, val_size=0.1, test_size=0.1):
    """
    Splits the dataset into training, validation, and test sets.
    
    Parameters:
    X (np.ndarray): Features dataset.
    Y (np.ndarray): Target dataset.
    train_size (float): Proportion of the dataset to include in the train split.
    val_size (float): Proportion of the dataset to include in the validation split.
    test_size (float): Proportion of the dataset to include in the test split.
    
    Returns:
    X_train, X_val, X_test, y_train, y_val, y_test: Splits for features and target.
    """
    
    # Ensure that the sum of the proportions is equal to 1
    if train_size + val_size + test_size != 1:
        raise ValueError("Train, validation, and test sizes must sum to 1.")
    
    # First split to carve out the training set
    X_temp, X_test, Y_temp, Y_test = train_test_split(X, Y, test_size=test_size, random_state=42)
    
    # Adjust validation size for the secondary split
    val_size_adjusted = val_size / (1 - test_size)
    
    # Second split to separate out the validation set
    X_train, X_val, Y_train, Y_val = train_test_split(X_temp, Y_temp, test_size=val_size_adjusted, random_state=42)
    
    return X_train, X_val, X_test, Y_train, Y_val, Y_test


In [85]:
sweep_config = {
    'method': 'random',  # Use random search
    'metric': {
        'name': 'loss',
        'goal': 'minimize'   
    },
    'parameters': {
        'booster': {
            'value': 'gbtree'
        },
        'objective': {
            'value': 'reg:squarederror'
        },
        'colsample_bytree': {
            'value': 0.7274556426850782
        },
        'learning_rate': {
            'value': 0.08645261544142283
        },
        'max_depth': {
            'value': 5
        },
        'alpha': {
            'value': 80
        },
        'n_estimators': {
            'value': 500
        },
        'grow_policy': {
            'value': 'lossguide'
        },
        'n_lag': {
            'values': [8, 24, 168, 336, 672]
        },
        'day_of_week_encoding': {
            'values': ['dummy', 'periodic']
        },
        'week_of_year_encoding': {
            'values': ['dummy', 'periodic']
        },
        'season_encoding': {
            'values': ['dummy', 'periodic']
        }
    }
}


In [86]:
def train():
    with wandb.init() as run:
        config = run.config

        # Create the forecasting dataset with the specified lags and encodings
        X, Y = create_forecasting_dataset(df_new, config.n_lag, 8, special_holidays,
                                          day_embedding=config.day_of_week_encoding, 
                                          week_embedding=config.week_of_year_encoding, 
                                          season_embedding=config.season_encoding)

        # Split the dataset into training, validation, and test sets
        X_train, X_val, X_test, y_train, y_val, y_test = split_dataset(X, Y)

        # Initialize the XGBoost model with the specified parameters from the sweep
        model = xgb.XGBRegressor(
            booster=config.booster,
            objective=config.objective,
            colsample_bytree=config.colsample_bytree,
            learning_rate=config.learning_rate,
            max_depth=int(config.max_depth),
            alpha=config.alpha,
            n_estimators=config.n_estimators,
            grow_policy=config.grow_policy,
            tree_method='gpu_hist',  # Change if not using GPU
            predictor='gpu_predictor'  # Change if not using GPU
        )

        # Fit the model
        eval_set = [(X_train, y_train), (X_val, y_val)]
        model.fit(X_train, y_train, eval_set=eval_set, verbose=True)

        # Evaluate the model
        mse_train, mape_train = evaluate_model(model, X_train, y_train)
        mse_val, mape_val = evaluate_model(model, X_val, y_val)
        mse_test, mape_test = evaluate_model(model, X_test, y_test)

        # Log the metrics to wandb
        wandb.log({
            "mse_train": mse_train, "mape_train": mape_train,
            "mse_val": mse_val, "mape_val": mape_val,
            "mse_test": mse_test, "mape_test": mape_test,
            "loss": mape_test  # This is the metric we want to minimize
        })


In [87]:
sweep_id = wandb.sweep(sweep_config, project='DI502_PROJECT_new')
wandb.agent(sweep_id, train)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: fcoz44e0
Sweep URL: https://wandb.ai/muratcankaplann98/DI502_PROJECT_new/sweeps/fcoz44e0


wandb: Agent Starting Run: pbofsl7f with config:
wandb: 	alpha: 80
wandb: 	booster: gbtree
wandb: 	colsample_bytree: 0.7274556426850782
wandb: 	day_of_week_encoding: dummy
wandb: 	grow_policy: lossguide
wandb: 	learning_rate: 0.08645261544142283
wandb: 	max_depth: 5
wandb: 	n_estimators: 500
wandb: 	n_lag: 8
wandb: 	objective: reg:squarederror
wandb: 	season_encoding: periodic
wandb: 	week_of_year_encoding: periodic
Exception in thread IntMsgThr:
Traceback (most recent call last):
  File "c:\Users\murca\anaconda3\envs\myenv\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\murca\anaconda3\envs\myenv\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\murca\anaconda3\envs\myenv\lib\site-packages\wandb\sdk\wandb_run.py", line 299, in check_internal_messages
Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "c:\Users\murca\anaconda3\envs\myenv\lib\threading.py", line 1016, in _bootstrap_

[0]	validation_0-rmse:1685.43121	validation_1-rmse:1668.60018
[1]	validation_0-rmse:1613.50769	validation_1-rmse:1599.52319
[2]	validation_0-rmse:1552.67447	validation_1-rmse:1541.06105
[3]	validation_0-rmse:1496.46205	validation_1-rmse:1486.65079
[4]	validation_0-rmse:1438.29364	validation_1-rmse:1430.40482
[5]	validation_0-rmse:1385.05759	validation_1-rmse:1379.46494
[6]	validation_0-rmse:1336.84910	validation_1-rmse:1332.91627
[7]	validation_0-rmse:1293.51731	validation_1-rmse:1291.13920
[8]	validation_0-rmse:1251.95779	validation_1-rmse:1251.77006
[9]	validation_0-rmse:1218.03225	validation_1-rmse:1219.17710
[10]	validation_0-rmse:1182.98175	validation_1-rmse:1185.80630
[11]	validation_0-rmse:1155.58728	validation_1-rmse:1160.11461
[12]	validation_0-rmse:1128.18508	validation_1-rmse:1134.50692
[13]	validation_0-rmse:1101.97893	validation_1-rmse:1109.68204
[14]	validation_0-rmse:1080.62495	validation_1-rmse:1089.91219
[15]	validation_0-rmse:1059.22263	validation_1-rmse:1069.39749
[1

loss,▁
mape_test,▁
mape_train,▁
mape_val,▁
mse_test,▁
mse_train,▁
mse_val,▁
loss,0.04136
mape_test,0.04136
mape_train,0.03037
mape_val,0.04056


...

[0]	validation_0-rmse:1650.70426	validation_1-rmse:1655.95090
[1]	validation_0-rmse:1543.97582	validation_1-rmse:1549.91834
[2]	validation_0-rmse:1446.04199	validation_1-rmse:1451.86083
[3]	validation_0-rmse:1355.29629	validation_1-rmse:1361.74738
[4]	validation_0-rmse:1272.99808	validation_1-rmse:1280.47441
[5]	validation_0-rmse:1199.46947	validation_1-rmse:1208.30542
[6]	validation_0-rmse:1133.42391	validation_1-rmse:1143.60425
[7]	validation_0-rmse:1072.94807	validation_1-rmse:1083.74532
[8]	validation_0-rmse:1016.54067	validation_1-rmse:1028.56586
[9]	validation_0-rmse:966.33266	validation_1-rmse:979.84704
[10]	validation_0-rmse:921.45030	validation_1-rmse:936.56740
[11]	validation_0-rmse:880.48482	validation_1-rmse:897.01571
[12]	validation_0-rmse:842.23652	validation_1-rmse:859.64122
[13]	validation_0-rmse:807.91874	validation_1-rmse:826.31098
[14]	validation_0-rmse:777.47524	validation_1-rmse:797.33004
[15]	validation_0-rmse:749.11442	validation_1-rmse:770.14653
[16]	validation_

loss,▁
mape_test,▁
mape_train,▁
mape_val,▁
mse_test,▁
mse_train,▁
mse_val,▁
loss,0.02346
mape_test,0.02346
mape_train,0.0131
mape_val,0.02386


wandb: Agent Starting Run: 4dhg1x0w with config:
wandb: 	alpha: 80
wandb: 	booster: gbtree
wandb: 	colsample_bytree: 0.7274556426850782
wandb: 	day_of_week_encoding: periodic
wandb: 	grow_policy: lossguide
wandb: 	learning_rate: 0.08645261544142283
wandb: 	max_depth: 5
wandb: 	n_estimators: 500
wandb: 	n_lag: 168
wandb: 	objective: reg:squarederror
wandb: 	season_encoding: periodic
wandb: 	week_of_year_encoding: periodic


C:\Users\murca\AppData\Local\Temp\ipykernel_2092\219453476.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=columns_to_drop, axis=1, inplace=True)
C:\Users\murca\AppData\Local\Temp\ipykernel_2092\219453476.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[float_cols] = df[float_cols].astype('float64')


[0]	validation_0-rmse:1651.48154	validation_1-rmse:1657.06716
[1]	validation_0-rmse:1543.82472	validation_1-rmse:1551.13763
[2]	validation_0-rmse:1444.94889	validation_1-rmse:1454.10275
[3]	validation_0-rmse:1354.93001	validation_1-rmse:1365.72570
[4]	validation_0-rmse:1272.30521	validation_1-rmse:1284.61912
[5]	validation_0-rmse:1198.32752	validation_1-rmse:1211.82297
[6]	validation_0-rmse:1130.51202	validation_1-rmse:1145.66259
[7]	validation_0-rmse:1068.75126	validation_1-rmse:1085.75606
[8]	validation_0-rmse:1014.43391	validation_1-rmse:1032.87346
[9]	validation_0-rmse:965.05483	validation_1-rmse:985.42132
[10]	validation_0-rmse:918.83813	validation_1-rmse:941.50466
[11]	validation_0-rmse:876.82699	validation_1-rmse:901.03182
[12]	validation_0-rmse:839.63470	validation_1-rmse:865.84437
[13]	validation_0-rmse:806.39730	validation_1-rmse:834.79521
[14]	validation_0-rmse:775.19887	validation_1-rmse:805.94361
[15]	validation_0-rmse:746.84923	validation_1-rmse:779.45238
[16]	validation_